In [ ]:
from functools import reduce
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)

In [ ]:
# OFFICES = ['U.S. House', 'State House']
OFFICES = ['U.S. House']
YEARS = ['2018', '2020', '2022', '2024']

TOP_N_FEATURES = 100
TOP_N_FEATURES_TO_DISPLAY = 15

census_datasets = [
    'b02001_race', 'b04007_ancestry', 'b05012_nativity_us', 'b08303_travel_time_work', 'b25003_housing_rentership', 
    'dp02_selected_social_characteristics', 'dp03_selected_economic_characteristics', 'dp04_housing_characteristics', 'dp05_age_race', 
    's0101_age_sex', 's1101_households_families', 's1201_marital_status', 's1501_educational_attainment', 's1701_income_poverty', 
    's1903_median_income', 's2101_veteran_status', 's2201_food_stamps', 's2301_employment_status', 's2401_occupation_sex', 
    's2403_industry_sex', 's2501_occupancy_characteristics', 's2701_health_insurance', 's2503_financial_characteristics',
]

drop_features = [
    'office_code', 
    'dem_share_prev','rep_share_prev', 'oth_share_prev', 
    'dem_share_change_prev', 'rep_share_change_prev', 'oth_share_change_prev', 
    'dem_votes_change_prev', 'rep_votes_change_prev', 'oth_votes_change_prev', 
    'registered_voters_change_prev', 'turnout_pct_change_prev', 
    # 'partisan_temp_prev', 'partisan_temp_change_prev', 
    'partisanship_lean_prev', 'partisanship_lean_change_prev', 'partisanship_lean_change_amount_prev',
]

In [ ]:
''' Remove top features not shared between 
    different datasets to prevent errors.'''
def removeUncommonColumns(nested_dict):
    print("Removing uncommon columns...")
    
    # Flatten and find common columns
    all_dfs = [df for year in nested_dict for df in nested_dict[year].values()]
    common_cols = set(all_dfs[0].columns)
    for df in all_dfs[1:]:
        common_cols &= set(df.columns)
    
    # Safely trim all dataframes
    for year in nested_dict:
        for office in nested_dict[year]:
            df = nested_dict[year][office]
            existing_cols = [col for col in common_cols if col in df.columns]
            nested_dict[year][office] = df[existing_cols]

    print('Done.')
    
    return nested_dict


''' Pull the engineered feature data along with its
    target for each year and office.'''
def makeDatasets(years, offices):
    print('Making datasets...')
    
    df_datasets = {}
    
    for year in years:
        print(f'Processing year {year}...')
        df_datasets[year] = {}
        
        for office in offices:
            office = office.replace(' ', '_').replace('.', '')
            print(f'Processing office {office}...')

            df = pd.read_csv('data/generated_data/07_ml_features_' + year + '_' + office + '.csv', low_memory=False)
            df_datasets[year][office] = df
    
    df_datasets = removeUncommonColumns(df_datasets)
    print('Done.')
    
    return df_datasets


''' Split features and target into X
    and y variables with some cleanup.'''
def makeFeaturesTargets(df):
    print(f'Making features and target...')
    
    y = df[['partisan_temp']]
    X = df.drop(columns=['standardized_id_num', 'partisan_temp', 'partisan_temp_change_curr'])
    X = X.replace(['-', '(X)', 'N/A', 'null', ''], pd.NA)
    
    X, y = X.align(y.dropna(), join='inner', axis=0)
    
    print('Done.')
    return X, y


''' Pipeline to impute and encode categorical
    variables, as well as scale, etc.'''
def fitModel(X, y):
    print(f'Fitting model...')

    categorical_cols = [
        'office_code',
        'partisanship_lean_curr',
        'partisanship_lean_prev',
        'partisanship_lean_change_prev',
    ]

    categorical_cols = [col for col in categorical_cols if col in X.columns]
    numeric_cols = X.select_dtypes(include=['number']).columns.tolist()

    numeric_cols = [col for col in numeric_cols if col not in categorical_cols]

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
    ])

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    preprocessor = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numeric_transformer, numeric_cols)
    ])

    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)

    print('Done.')

    return model, X_train, X_test, y_train, y_test, numeric_cols


def makePredictions(X_test, model):
    print(f'Making predictions...')
    y_pred = model.predict(X_test)
    print('Done.')
    return y_pred


def plotAccuracy(y_test, y_pred):
    print(f'Plotting accuracy...')
    plt.figure(figsize=(12, 9))
    plt.scatter(y_test, y_pred)
    plt.xlim(-1.25, 1.25)
    plt.ylim(-1.25, 1.25)
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("Prediction Accuracy")
    plt.grid(True)
    print('Done.')
    return plt


def featureCoeff(model):
    print(f'Computing feature coefficients from pipeline...')

    regressor = model.named_steps['regressor']
    preprocessor = model.named_steps['preprocessor']
    coef = regressor.coef_.flatten()

    # Inline feature name extraction
    output_features = []
    for name, transformer, columns in preprocessor.transformers_:
        if transformer == 'drop' or transformer is None:
            continue
        if hasattr(transformer, 'get_feature_names_out'):
            try:
                names = transformer.get_feature_names_out(columns)
            except:
                names = columns
        else:
            names = columns
        output_features.extend(names)

    feature_names = output_features

    if len(coef) != len(feature_names):
        raise ValueError(f"Mismatch: {len(coef)} coefficients vs {len(feature_names)} feature names")

    df = pd.DataFrame({
        'feature': feature_names,
        'coefficient': coef,
        'abs_coefficient': np.abs(coef)
    })

    top_features = df.sort_values(by='abs_coefficient', ascending=False).head(TOP_N_FEATURES_TO_DISPLAY)
    print('Done.')
    return top_features


def plotFeatureCoeff(features):
    print(f'Plotting feature coefficients...')
    plt.figure(figsize=(12, 18))
    bars = plt.barh(features['feature'], features['coefficient'])
    plt.xlabel('Coefficient Value')
    plt.xticks(fontsize=7)
    plt.yticks(fontsize=7)
    plt.title(f'Most Influential Features (Linear Regression)')
    plt.axvline(x=0, color='gray', linestyle='--')
    plt.grid(True, axis='x', linestyle=':', alpha=0.7)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    print('Done.')
    return plt


def get_feature_names(model):
    print("Getting feature names...")

    preprocessor = model.named_steps['preprocessor']
    feature_names = preprocessor.get_feature_names_out()

    # Store both prefixed and cleaned names
    cleaned_names = [name.split('__', 1)[-1] for name in feature_names]

    print(f"Done. Retrieved {len(feature_names)} feature names.")
    return feature_names, cleaned_names


def mergeTopFeatures(top_features_lists):
    print(f'Creating common top features using clean names...')
    from itertools import chain

    # Make clean names
    normalized_lists = []
    for item in top_features_lists:
        if isinstance(item, list):
            normalized_lists.append(item)
        elif hasattr(item, 'columns') and 'feature' in item.columns:
            normalized_lists.append(item['feature'].tolist())
        else:
            raise ValueError("Each item must be a list or a DataFrame with a 'feature' column")

    # Find intersection
    common_features = set(normalized_lists[0])
    for feature_list in normalized_lists[1:]:
        common_features.intersection_update(feature_list)

    # Preserve order
    seen = set()
    merged_common_ordered = []

    for item in chain.from_iterable(normalized_lists):
        if item in common_features and item not in seen:
            seen.add(item)
            merged_common_ordered.append(item)

    print('Done.')
    return merged_common_ordered


def one_hot_encode_selected(df, columns_to_encode):
    df = df.copy()
    
    if not columns_to_encode:
        return df
        
    encoded = pd.get_dummies(df[columns_to_encode], prefix=columns_to_encode)
    df = df.drop(columns=columns_to_encode)
    
    return pd.concat([df, encoded], axis=1)

In [ ]:
def getRankedFeatureList():
    df_ranked_features = pd.read_csv('data/generated_data/df_importances.csv')
    df_ranked_features = df_ranked_features.sort_values(by='Average', ascending=False)
    df_ranked_features = df_ranked_features[~df_ranked_features['Feature name'].isin(drop_features)]
    
    df_ranked_features_top = df_ranked_features.head(TOP_N_FEATURES)
    
    ranked_features_top_list = df_ranked_features_top['Feature name'].tolist()
    
    return ranked_features_top_list

ranked_features_top_list = getRankedFeatureList()

In [ ]:
def predDatasetsIndiv(df_datasets, years, offices):
    for year in YEARS:
        print(f'Processing year {year}...')
        
        for office in OFFICES:
            office = office.replace(' ', '_').replace('.', '')
            print(f'Processing office {office}...')
            
            df = df_datasets[year][office].copy()
            X, y = makeFeaturesTargets(df)
            X = X[ranked_features_top_list]
    
            print(f"Training over {len(X.columns)} features...")
            
            model, X_train, X_test, y_train, y_test, numeric_cols = fitModel(X, y)
            y_pred = makePredictions(X_test, model)
            
            mse = mean_squared_error(y_test, y_pred)
            print("Mean Squared Error:", mse)
    
            r2 = model.score(X_test, y_test)
            print("R2 Score:", r2)
    
            plt = plotAccuracy(y_test, y_pred)
            plt.savefig(f'output/figures/regression_accuracy_{year}_{office}_individual.png')
            # plt.show()
            plt.close()
            
            top_features = featureCoeff(model)
            plt = plotFeatureCoeff(top_features)
            plt.savefig(f'output/figures/regression_top_features_{year}_{office}_individual.png')
            # plt.show()
            plt.close()

#### Predict Individual Datasets

In [ ]:
years = ['2018', '2020', '2022']
offices = ['U.S. House']

df_datasets = makeDatasets(years, offices)

predDatasetsIndiv(df_datasets, years, offices)

#### Fit/Train Final Model
Do we have data leakage here? Maybe not, if we train on historical data and
<br>run a separate test on newer data in a following step. Say, here we
train 2018-2022, <br>and then in another cell test 2024 on the same model

In [ ]:
def aggDatasets(datasets, years, offices):
    dfs = []
    
    for year in years:
        print(f'Processing year {year}...')
        for office in offices:
            office = office.replace(' ', '_').replace('.', '')
            
            print(f'Processing office {office}...')
            dfs.append(df_datasets[year][office].copy())
            
    df = pd.concat(dfs, axis=0, ignore_index=True)
    
    return df

In [ ]:
def fitDatasetsAgg(df_datasets):
    X, y = makeFeaturesTargets(df)
    X = X[ranked_features_top_list]

    print(f"Training over {len(X.columns)} features...")
    
    model, X_train, X_test, y_train, y_test, numeric_cols = fitModel(X, y)
    y_pred = makePredictions(X_test, model)

    expected_columns = X.columns.tolist()
    
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", mse)

    r2 = model.score(X_test, y_test)
    print("R2 Score:", r2)

    plt = plotAccuracy(y_test, y_pred)
    plt.savefig(f'output/figures/regression_accuracy_aggregate.png')
    plt.show()
    # plt.close()
    
    top_features = featureCoeff(model)
    plt = plotFeatureCoeff(top_features)
    plt.savefig(f'output/figures/regression_top_features_aggregate.png')
    plt.show()
    # plt.close()

    return model

In [ ]:
def predDatasetsAgg(df_datasets):
    X, y = makeFeaturesTargets(df)
    X = X[ranked_features_top_list]

    print(f"Training over {len(X.columns)} features...")
    
    model, X_train, X_test, y_train, y_test, numeric_cols = fitModel(X, y)
    y_pred = makePredictions(X_test, model)
    
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", mse)

    r2 = model.score(X_test, y_test)
    print("R2 Score:", r2)

    plt = plotAccuracy(y_test, y_pred)
    plt.savefig(f'output/figures/regression_accuracy_aggregate.png')
    plt.show()
    # plt.close()
    
    top_features = featureCoeff(model)
    plt = plotFeatureCoeff(top_features)
    plt.savefig(f'output/figures/regression_top_features_aggregate.png')
    plt.show()
    # plt.close()

    return model

#### Fit aggregated data
This functionality produces the <code>model</code> object to be used later. Make sure no datasets from this
<br>cell is NOT included in holdout testing.

In [ ]:
years = ['2018', '2020', '2022']
offices = ['U.S. House']

df_datasets = makeDatasets(years, offices)
df = aggDatasets(df_datasets, years, offices)

X, y = makeFeaturesTargets(df)
X = X[ranked_features_top_list]
X = X.replace({pd.NA: np.nan})

model, X_train, X_test, y_train, y_test, numeric_cols = fitModel(X, y)
y_pred = makePredictions(X_test, model)

expected_columns = X.columns.tolist()

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

r2 = model.score(X_test, y_test)
print("R2 Score:", r2)

plt = plotAccuracy(y_test, y_pred)
plt.savefig(f'output/figures/regression_accuracy_aggregate.png')
plt.show()

top_features = featureCoeff(model)
plt = plotFeatureCoeff(top_features)
plt.savefig(f'output/figures/regression_top_features_aggregate.png')
plt.show()

#### Holdout Prediction
This functionality requires the <code>model</code> object from a previous cell. Make sure this holdout
<br>dataset was not included in the model's training.

In [ ]:
years = ['2024']
offices = ['U.S. House']

df_datasets = makeDatasets(years, offices)

X, y = makeFeaturesTargets(df)
X = X[ranked_features_top_list]

# Scale X
X = X.replace({pd.NA: np.nan})

# Expected columns from training
# for col in expected_columns:
#     if col not in X_holdout.columns:
#         X_holdout[col] = np.nan  # or 0
# X = X[expected_columns]

# NaN missing cells
X = X.replace({pd.NA: np.nan})

# # Predict, transformations handled by pipeline
y_pred = model.predict(X)

# MSE
mse = mean_squared_error(y, y_pred)
print("Mean Squared Error:", mse)

# R2
r2 = model.score(X, y)
print("R2 Score:", r2)

plt = plotAccuracy(y, y_pred)
plt.savefig(f'output/figures/regression_accuracy_holdout.png')
plt.show()

top_features = featureCoeff(model)
plt = plotFeatureCoeff(top_features)
plt.savefig(f'output/figures/regression_top_features_holdout.png')
plt.show()

### Grid search

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score

def gridSearch(df):
    df = df.copy()
    X, y = makeFeaturesTargets(df)
    X = X[ranked_features_top_list]
    X = X.replace({pd.NA: np.nan})
    
    numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
    
    models = {
        'LinearRegression': (
            LinearRegression(), 
            {}
        ),
        'Ridge': (
            Ridge(),
            {'regressor__alpha': [0.01, 0.1, 1.0, 10]}
        ),
        'Lasso': (
            Lasso(max_iter=10000),
            {'regressor__alpha': [0.01, 0.1, 1.0, 10]}
        ),
        'ElasticNet': (
            ElasticNet(max_iter=10000), 
            {
            'regressor__alpha': [0.01, 0.1, 1.0],
            'regressor__l1_ratio': [0.2, 0.5, 0.8]
            }
        ),
        'DecisionTree': (
            DecisionTreeRegressor(), 
            {
            'regressor__max_depth': [5, 10, None],
            'regressor__min_samples_split': [2, 10]
            }
        ),
        'RandomForest': (
            RandomForestRegressor(n_jobs=-1), 
            {
            'regressor__n_estimators': [50, 100],
            'regressor__max_depth': [5, 10, None]
            }
        ),
        'GradientBoosting': (
            GradientBoostingRegressor(), 
            {
            'regressor__n_estimators': [50, 100],
            'regressor__learning_rate': [0.05, 0.1],
            'regressor__max_depth': [3, 5]
            }
        ),
    }
    
    y = y.values.ravel() 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    best_models = {}
    results = []
    
    for name, (model, params) in models.items():
        print(f"🔍 Tuning: {name}")
        
        pipe = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', model)
        ])
        
        grid = GridSearchCV(pipe, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
        grid.fit(X_train, y_train)
    
        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)
    
        mse = mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
    
        results.append({
            'Model': name,
            'Best Params': grid.best_params_,
            'MSE': mse,
            'R2': r2
        })
    
        best_models[name] = grid 
    
    results_df = pd.DataFrame(results).sort_values(by='R2', ascending=False)

    return best_models, results_df

In [ ]:
years = ['2018', '2020', '2022']
offices = ['U.S. House']

df_datasets = makeDatasets(years, offices)
df = aggDatasets(df_datasets, years, offices)

best_models, results_df = gridSearch(df)

In [ ]:
def fitBestModel(df, params):
    df = df.copy()
    
    X, y = makeFeaturesTargets(df)
    X = X[ranked_features_top_list]
    X = X.replace({pd.NA: np.nan})
    
    numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
    
    gbr_final = GradientBoostingRegressor(
        random_state=42,
        **params
    )
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', gbr_final)
    ])
    
    model.fit(X, y.values.ravel())

    return model

In [ ]:
def cleanParams(best_models, model_name):
    params = best_models[model_name].best_params_
    clean_params = {k.replace('regressor__', ''): v for k, v in params.items()}
    
    print(f'Cleaned params for {model_name}: {clean_params}')
    
    return clean_params

In [ ]:
years = ['2018', '2020', '2022']
offices = ['U.S. House']

df_datasets = makeDatasets(years, offices)
df = aggDatasets(df_datasets, years, offices)

clean_params = cleanParams(best_models, 'GradientBoosting')
model = fitBestModel(df, clean_params)

In [ ]:
def makePredsBestModel(df, model):
    df = df.copy()
    
    X, y = makeFeaturesTargets(df)
    X = X[ranked_features_top_list]
    X = X.replace({pd.NA: np.nan})
    X = X.dropna()
    
    # Align datagrame to cleaned X
    df = df.loc[X.index]
    
    y_pred = model.predict(X)
    
    y_true = df["partisan_temp"]
    
    df["predicted_partisan_temp"] = y_pred
    
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")

    return df, y_pred, y_true

In [ ]:
years = ['2024']
offices = ['U.S. House']

df_datasets = makeDatasets(years, offices)
df_holdout = df_datasets['2024']['US_House']

df_pred_best, y_pred_best, y_true_best = makePredsBestModel(df_holdout, model)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_true_best, y_pred_best, alpha=0.4)
plt.plot([-1, 1], [-1, 1], color='red', linestyle='--')  # Diag. line
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Predicted Temperature vs. Actual Temperature")
plt.grid(True)
plt.tight_layout()
plt.show()